In [1]:
# set dependencies
import os
import csv
import requests
import time
import psycopg2
import pymongo

import pandas as pd

from flask_pymongo import PyMongo
from splinter import Browser
from bs4 import BeautifulSoup as bs
from flask import Flask, render_template, redirect


pd.options.display.float_format = '{:,.2f}'.format
pd.options.display.max_rows=50

# National Parks Visitor table

In [2]:
# Get data table
tables=pd.read_html('https://www.nps.gov/aboutus/visitation-numbers.htm', index_col=0)
table=pd.DataFrame(tables[1])
table

,Park,Recreational Visits
1,Great Smoky Mountains National Park,12.1 million
2,Yellowstone National Park,"3,8 million"
3,Zion National Park,3.6 million
4,Rocky Mountain National Park,3.3 million
5,Grand Teton National Park,3.3 million
6,Grand Canyon National Park,2.9 million
7,Cuyahoga Valley National Park,2.8 million
8,Acadia National Park,2.7 million
9,Olympic National Park,2.5 million
10,Joshua Tree National Park,2.4 million
